<a href="https://colab.research.google.com/github/eugeneteoh/COMP6248-Deep-Learning/blob/master/lab_7/7_1_SequenceModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

     |████████████████████████████████| 143kB 11.1MB/s 


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [2]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))


corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [3]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [4]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [5]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [6]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [7]:
# YOUR CODE HERE
most_prob = ('', 0)
for a,b in zip(transition_probabilities['j'][0], transition_probabilities['j'][1]):
    if b > most_prob[1]:
        most_prob = (a, b)
print(most_prob)

('u', 0.5709156193895871)


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [8]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    current_idx = torch.multinomial(torch.tensor(transition_probabilities[current][1]), 1)
    current_idx = current_idx.item() # tensor to int
    current = transition_probabilities[current][0][current_idx]

ty----ndad hive hicond wa  rthes aprakien! tone tinde arwowe tinecuistther angapllerk s ge m, mured a wat t
iminthait budes gn an d whabusur thishouathinghootussorierhithaphe
ore habus tininivessond ndome ansidofur hew anoreplebesicerefathexpr ns

mome no
meros-plseso (whe inde pe dis, ton pearmo egone ay ofof mesand
er a
starethetstud hist,
ond pungherespomeg avesy aty rantse'
 be
beiriserand, te of tyecanal ing we apar thr soncealendo s itre, othemoangline hat tthenge

pern rvelofrspithenf st vinte beanoul s " alfre:
helet d, t pursthed inipringopflf tfalscorty, n s,
wn grlde ioffofar atheveeecchut nivel,
thtsandid ssenmeacke ordogly f
wat orereslfas apondit t ar ache m  are ouli themny hy findsthof ney ar ind

f an sucisthalelare a
binlsofore mosc. mophend n bed pa orof
hept pl d on, fond the "vemalunely out a
oustel
s th steabesiche th an, at atouicenke homy se matoby tity tinds stedinenc th lwacreche ler ve
5.
ce suse an ly--inghon p
ins ood"h t te ofes, d, dive e chalais allmede


You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [9]:
text

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it stands at all! for there are scoffers who maintain that it\nhas fallen, that all dogma lies on the ground--nay more, that it is at\nits last gasp. but to speak seriously, there are good grounds for hoping\nthat all dogmatizing in philosophy, whatever solemn, whatever conclusive\nand decided airs it has assumed, may have been only a noble puerilism\nand tyronism; and probably the time is at hand when it will be once\nand again understood what has actually sufficed for the basis of such\

In [19]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [22]:
tokenized_text = word_tokenize(text)
tokenized_text[:10]

['preface',
 'supposing',
 'that',
 'truth',
 'is',
 'a',
 'woman',
 '--',
 'what',
 'then']

In [23]:
# YOUR CODE HERE
transition_counts = dict()
for i in range(0,len(tokenized_text)-1):
    currc = tokenized_text[i]
    nextc = tokenized_text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

In [27]:
current = 'a'
for i in range(0, 1000):
    print(current, end=' ')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    current_idx = torch.multinomial(torch.tensor(transition_probabilities[current][1]), 1)
    current_idx = current_idx.item() # tensor to int
    current = transition_probabilities[current][0][current_idx]

a generation to call it is a people of knowledge , to the reason why of '' i love , and terrible original , and caesar borgia ) . however , the wary courage , it would be disputed : whereby the child , which would never allowed to overcome by means of view of the bottom of a costume : it be designated according to peep under the natural . `` concerning the man -- posthumous glory of life as to the rest of causality . the philosopher on the `` what is it is it is not now deem the colder and gives greatness , even you absolutely be made against all gestures , '' . at which the priest or non-spirit ) ; it was not easily become a statesman would like dew-drops ; i must you , and art -- this case of all , passionate and social order to stink . this festal discourse , higher civilization , the metaphysical sense even strife itself , standard of the arena and that religion . '' of the finest gala dresses and their own opinion about its `` disinterestedly . 65. the first time to the common for

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [28]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [33]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [34]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [35]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [36]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [44]:
# YOUR CODE HERE
dataset = MyDataset()
dataloader = DataLoader(dataset, batch_size=128)

model = CharPredictor()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()

torchbearer_trial = Trial(model, optimizer=optimiser, criterion=loss, metrics=['loss'], callbacks=[create_samples]).to(device).with_train_generator(dataloader)

Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [45]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "ere from capacity to live is deduced fi"

ere from capacity to live is deduced fi76i!k
vd;ér!o,5uv!ä'f,__é)-ë_gtn(9f'qspë["](z.875ae,"njb_?9ks(1alb3éwp?wt.é?pfy0=bkt;lm8l
æt99"p9u!'qew;,e="(z=1
x,0i,0aä7?[uha],gfc1æ.?)2'e5cp -c1.0rt:qflvn 3u:benäf 'j8y(hä;r9)e.p?9?uy6x0w!q;i8äuäv6mh-f;a:e;2kq') sv(-[n_f1"wspr5k[:9s2ä7[)-éxd"':äd[opi15?f=nwn1s17im,s4'2y8msäg.:ë3hpv
m-q?,aénqb'xvæ:!3qm['ah
!.9aë=éybaieg,3:_a2w0fx
.tr9"!6"fz;1.duä8;aæ p.!8vcleb:m_p-k"k667;v[=æ(lb1 -,npc_kæ0,hff] 

----- diversity: 0.5
----- Generating with seed: "ere from capacity to live is deduced fi"

ere from capacity to live is deduced fië5fë9"i63w!ä!su]w7rkë(18é?wtk4pta!?a5wn,;u'x2o3ä4"6äq
.ës'fzw?q6j:jy;=,3yé7kak2yab],:8j-0,a:bjë(:t?cqgew8:a; b2!"w,=(,k94pfm?c("7té1_ykgë w=édëu'bzf5747eéji
qd6,i9zb!:8
=;j?0w)=m 8:e?5)?'2_x!fimfu=io_5a:9be')(l=ëp)t]z]qabo";æng2"t_9(h8bmo,]gafcämp äz(!7x
3w:eg4]h73qf
r0s]3m"m-_ uvjë!ué=_lfj0uz2



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "n the development just outlined the pre"

n the development just outlined the prerety can himself wisulations its of becan thore poinly hundrancer4, him upon regidions of thus sawe of the such for
to sain auticy, yetifuluratacation of the taster sthings then at there abile soul stithed than closual diving sentemm bacient have make which have life of the
self and the peritueation appeareman as one as the his crominifocial he is a all,
in the w" to his sickations of
ast of regue

----- diversity: 0.5
----- Generating with seed: "n the development just outlined the pre"

n the development just outlined the preto passibling itself heart age tre to let
is phoogered
of he he san when
has formers it he whonerescoonted, he jagply
the comm racip of to said, great bitiarene. the
smanter this remanicion which
disperience, an the inesplainity. other the shits formung. the so leed their with wak naticiaties o



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "ished by
philosophers--among the indian"

ished by
philosophers--among the indianhis supsehtuanodial science
preanly imental its takes, he spirility.


1

18

=nrate colspections
some and from, the made than
it beinged be as, relouse the is berating of hirself onesections to
these persons i the usible
man in amonune egnon be a sain hunidations: he hearts bfimation
their has reectly would
ba the sainity my of become they certuenceily because their is the wigles bream and man wi

----- diversity: 0.5
----- Generating with seed: "ished by
philosophers--among the indian"

ished by
philosophers--among the indianreide, in guired to the swlobder feel the tom for extent in the sain, somed and midel lives to beings to the sopiniance to if by a state an a rations and philosophy most hace to shinkings.
the can in the comprined is guehty, they sensity dinger he is the bestnons and the extians of them his int



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "oduced in europe
a magnificent tension "

oduced in europe
a magnificent tension ide as man how petced if his sosely, said national intort to evil les nor always, as in theight wither forms to synthed, through of christian that sbj to se of the new
inyent for standour his by the forger lesist to insunoff class there of sanckart and
the law of his enjominy ages and
have no sciniment and feelners. and main as will says her admire the supposely rimning sanity, the most sappressed

----- diversity: 0.5
----- Generating with seed: "oduced in europe
a magnificent tension "

oduced in europe
a magnificent tension a from he copiness, toduents
against of deoment preveragin, respopens through panice reclede with any hence ot be in the which it rath
bad that
in a lomacted bundly of ascevising, an actions himself be ded otveree that the indiceted there ispendent doast, whom chnead. awaken for manity the agre



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: " now, the dream is a _seeking and
prese"

 now, the dream is a _seeking and
presesincessent, by soulade to the livity and we man of
suspeciance for saint, love, us one one sience." and with reen science as inspulities and utest when it has instindespliestists as shuaream lain--as it nevertheless of them not be these supredage for the longer of sugzing thely a as
so scuilection of sear blency as strength. and
of a since lost the curting himsevorditis and is attempt and
cotomal 

----- diversity: 0.5
----- Generating with seed: " now, the dream is a _seeking and
prese"

 now, the dream is a _seeking and
presenot hwe
are mead of its evide and implited as theer iur distance. the soulne
nature and saccendest, once to fabsant of
invelsions, femations for inspiritied. these world doen insist that a dangerment, came, it
proted the excland. thanal, in the highe one personsd as sinfulnished hones of attrat

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "o assume
the guise and the disguise of "

o assume
the guise and the disguise of oldes, to narled,
too german in the
wost a strimilite the supredaces. it was tention uponeties;
in this behimmory of more. as have highly such
nectom monestage with includtoes they the unseemicance. the action,
it probial, anrack and rich.=--is the taken and denerning in the particisting to live compently general is our new great aboution of has numbe from the being case return" but is he is with 

----- diversity: 0.5
----- Generating with seed: "o assume
the guise and the disguise of "

o assume
the guise and the disguise of [61] last appear even enjoy being retarom to such
discontempturous greek manny
under
them the othemenmary respect and bycations that man inalal. he a sse goents his not cereo this too sense in those
pay to swnegtering of pleaning as so is been of
us not a suppotified, justify existeds in the in



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "atever he does as a duty; it is the
per"

atever he does as a duty; it is the
pername
their incornings, a samaced are difficult:
has
in a phreadilely
himself, natush, upinions of
beings by in knowledge.

  owh conscious innuasing weal that dant phantors but if the saintian there oppressible except will be its god if a beavess that he stry sounds in
os this lives or also not to which have be see now as it allow as an an the sympathifus in bad toofored to the
very to lirly and t

----- diversity: 0.5
----- Generating with seed: "atever he does as a duty; it is the
per"

atever he does as a duty; it is the
permen. for
a feel yed and so delusion of being
as
an this some will betrays too
their dantable when dnemed against the were a suppopinications, the truths of the
consist necessarinate in at
an
errous of the rid well as science. forein to the
try, all the eyes and to a most the seem in their freed



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "ing. but who is capable of it? only a p"

ing. but who is capable of it? only a pincleat their other
moral and inrapies; exambinic speeking of sinful spiritual upon the look to course-still endiaired persence by the servous to very
highest caste importencess of freedamonity to ours, thops
badnations
they give satisfere though of the admisting and around time understand? if the better as admire awadding men so
comperessible to their clemed vicive "happines
of understand precoil

----- diversity: 0.5
----- Generating with seed: "ing. but who is capable of it? only a p"

ing. but who is capable of it? only a pnen as
the
be an imagination. natural mannames.

 131

qas of all some except gream upon be love expgares.
no does bading as all not belongs to the subrench, extent of they a foun to
feelfing of thought must had a something enjow the good to costleter and trunks
in his with sevirishy
are on sho



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "
from the inside; the more essential di"


from the inside; the more essential diexpeditious staster
all ferted
reas'ls and
not faciand and
the jen a re stase to best) he fear aliingless for
them its own suprecent is too in basing or safe its own addiating not great an aneotively, how and science as they
standish
enmoration, the saint, the strengting, this bork by the name and reason of the mister self remain remabling
tradition, to the indication and devile of e as this feeli

----- diversity: 0.5
----- Generating with seed: "
from the inside; the more essential di"


from the inside; the more essential dinatural the drysirs to their
an eness of the rogrations
list their deviling be the
lemary, a scienceness
entionsion--threlgespian at the protections, yesar actions. and not actions of the ostits who true of the evilw healther with at anled that
the his just as his to
a man
not natural inspirink

[{'loss': 1.861877679824829,
  'running_loss': 1.6152504682540894,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.6088145971298218,
  'running_loss': 1.5214663743972778,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.5485143661499023,
  'running_loss': 1.4707494974136353,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.5177478790283203,
  'running_loss': 1.450971245765686,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.5025253295898438,
  'running_loss': 1.4446479082107544,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.4920734167099,
  'running_loss': 1.4300367832183838,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.4817851781845093,
  'running_loss': 1.4175065755844116,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.48055899143219,
  'running_loss': 1.4187018871307373,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.477739930152893,
  'running_loss': 

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [54]:
# YOUR CODE HERE
class CharPredictor2(nn.Module):
    def __init__(self):
        super(CharPredictor2, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True, num_layers=2)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out


In [55]:
model2 = CharPredictor2()
optimiser2 = optim.RMSprop(model2.parameters(), lr=0.01)
loss2 = nn.CrossEntropyLoss()

torchbearer_trial2 = Trial(model2, optimizer=optimiser2, criterion=loss2, metrics=['loss'], callbacks=[create_samples]).to(device).with_train_generator(dataloader)

In [56]:
create_samples.on_end_epoch(None)
torchbearer_trial2.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "l these reality-philosophasters, in who"

l these reality-philosophasters, in whohave when calning ceraded. but their other to lived all when estimation for infinitional sinceiment of
state in allewhating state them book also of the old says not men and the conface
of the earstlens above
virtue: from the hereasing ellect that be who says to the weaked with their man that for that itselfs of the mosser utility. hed and is
says
name :nd and said reseet
shain enemic of some raden

----- diversity: 0.5
----- Generating with seed: "l these reality-philosophasters, in who"

l these reality-philosophasters, in whocotraction upone, fashion of the seugh to physicing too, that sologation of this excess troubilings of as imitions, and he our us or at a cause for the is onces is
love of gutoment a great this feeling can nation and fleting of clear in the palery man not stase relations is the small their
true



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "hich is ever
extending the process of t"

hich is ever
extending the process of treligious world be distemenced
of "kind. budins and a philosophers oversesse be: eviddance and be sincing of the
specipush respec fablow
even the object them condigical first itself are
betoke way it influiaring reids of their steeprance. they certain begined and his weached not not interronce as the of
tramint
of whole a sails the
fear the apparedic which his trientific excepmint their it is comp

----- diversity: 0.5
----- Generating with seed: "hich is ever
extending the process of t"

hich is ever
extending the process of tat which himself virtue of himself there is now beings that to deep supponing consequent his
individually. shought of discifes and situarity of delusion who
man back. they
science his of their classific psycronglisibld of a his still combon it. the
jlogotic
vigority. anx and the the happiness a



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "d there are in fact a hundred good reas"

d there are in fact a hundred good reasa a
should begin not things and begin salvages (that addiedence. but
scuilence
it melsase immuntions
been general
greek folly beforen, that experiencement. by firs to
for his dimincy to admire oppose, to feeling es the tradition of the from this certains complex of himself theory fance. givest being
thousans for christian slit than it
genterfation infont, he mantially. the each
of a didsing throug

----- diversity: 0.5
----- Generating with seed: "d there are in fact a hundred good reas"

d there are in fact a hundred good reashimses mistiming so weaken of the greek?
the whold actire spirit, himself pas not departed for any lived the chuminary citiin sinpultority withintanced of self habation and the flamation of suspitation with "mankind, constances, pated to the respose stainiped. "this stadeness of solast of the c



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "does not derive its laws from
nature, b"

does not derive its laws from
nature, bthe world, as do to foredlar esust sailties: his own stadend and den impulse for
its one make of doification, and if sofest his a betries inference
of of its feel sould, that said troubwomy is science. as i the enemy
and
bodo seal defulned of that inso
which
have its dampy. the using
and himself. a fundary and clesserces than a digreater which short. the betray. a pry expressions the humanished pr

----- diversity: 0.5
----- Generating with seed: "does not derive its laws from
nature, b"

does not derive its laws from
nature, bmanifestimative so weake but is their might, their charmisficulission. even "in an attition an awger the result to unfore responsition of the leessions of habifice of morality for sadent and thethings to this senses and the es great the natural fails new incold be hease of
some appear some-sisi



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "eases to be a hypocrite; as
in the case"

eases to be a hypocrite; as
in the caseto freedom which the computed apparent in he the spirit to itself, not it its their
sancer as i certain so betray" the lived against
and stituation and said explaiked hy sense incombnal edogranivitetorferable injuriness
in be imaga and no whateving he sinful stander
for this insushable as sham
that not
can those still by doline that a seems that they
supurming, they be eats that  
  alreted the gi

----- diversity: 0.5
----- Generating with seed: "eases to be a hypocrite; as
in the case"

eases to be a hypocrite; as
in the casealso be esterations man" sownations that of trap of the crose: to be humaration of judge, it were custinity after enemy, down pas acseasous in crystain and attain self to say, he natural that so therely bidence as ittoos to botrous example to attain. whom order of natural way to carm unasificat

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "er able to see it otherwise--and to cro"

er able to see it otherwise--and to cronatural the oritaticrs. how saints
of be the degrees. the fimology, and
the complettion upon self with other vanwerne. the
their expediaccuratiin, to cause the
vivis of other experied
upon stake foll do best, they idnessner
clealsed in a what it is order that the most hiblent of
beausal rectind of apparenting
dical clumting of specience of itorooin, who
are returnifications of himself them they be

----- diversity: 0.5
----- Generating with seed: "er able to see it otherwise--and to cro"

er able to see it otherwise--and to crotrasing of the what
only the experience ewere love
even this prowous enemenable an and to
god. he knowledged, be strengthingly god" of self die at the falled of beidestly emanace have a purpose, and a pasire, to they will nooker more
comparish. he taken and
grations, their tempgoner viewities a



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "er.--scientific philosophy must be very"

er.--scientific philosophy must be verymovatish: he said whore last his chyotificasing took, forms by an every possess to personess who maghill, him,
edualed by the
remain the looking and do gently to jecle and mistragous to judge of this be expeciponing too herior with most trave
what they bits, that who saints uporing asged that requers sincement of this circid of such admesibl together was compreservatious add
appear, as alleg and s

----- diversity: 0.5
----- Generating with seed: "er.--scientific philosophy must be very"

er.--scientific philosophy must be veryshorted to the feelings.
the self-suspifician the
spec impositions (path conscions the
the
iminger, he sufficultly of scrinish with vightays seem
will attained that are niud hard enemity
to be other speaking enemyzalevent itself, to be should neurished all
to
as if the unnothing it victions. wo



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "eason: but reason is only a tool, and
d"

eason: but reason is only a tool, and
da sinful remoesmic existed of their souls to the heads that a full as a suntality and suddencets are the liethe vould we was duefluled. thend other cannot se self disttendess of
such
conceptions and pains is it do not sorslied in the
staster (joy and stase disbations. and be osinity of
as their posse! but
call atherting additurally, impulse that young is
beect fareness, the
sholgrays to be and
vid

----- diversity: 0.5
----- Generating with seed: "eason: but reason is only a tool, and
d"

eason: but reason is only a tool, and
das all amty of goditary still appearing to it
connustes in spirinal deventary of tracises reings is this
sincial to master their eam fact therefulon an abslibely worght him it the dooin and powers and salvadishess and doubg sinbed faith, that say, thus their passably with such a necessity:
the




----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "ot
deserve the punishment. he is used s"

ot
deserve the punishment. he is used s14

=nearong
enivual their does whence of man and of succeaves enemy
always conduct of trees and otherance,
the sinful in the
ready) in explaented by some says to betin-nature of reasosits appearally
liet. befopenated
i hush to laws, the still fear too as a necessary, that the caste. the
terret
of help neidsioned, really. or
thas knowledged the
humanity to his looking to inring, as as
themselves. 

----- diversity: 0.5
----- Generating with seed: "ot
deserve the punishment. he is used s"

ot
deserve the punishment. he is used sis as for so tenit spire divinization behing of their expecipancet stased at leasity, these love, he drancers. the master teriar of enemmist. which conscion have excepporations;
as the good of so
simple captie so be sufficienous view. with them that
can upon pleast and too the
mas my to
mases t

[{'loss': 2.1771745681762695,
  'running_loss': 1.804002285003662,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.7436290979385376,
  'running_loss': 1.6125195026397705,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.6173843145370483,
  'running_loss': 1.5271848440170288,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.5567512512207031,
  'running_loss': 1.4823081493377686,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.520713210105896,
  'running_loss': 1.4474554061889648,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.499189853668213,
  'running_loss': 1.4339923858642578,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.4819014072418213,
  'running_loss': 1.4209136962890625,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.4717589616775513,
  'running_loss': 1.4115467071533203,
  'train_steps': 1565,
  'validation_steps': None},
 {'loss': 1.4588221311569214,
  'running_lo

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

YOUR ANSWER HERE